In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as pyplot
import geopandas

# Handling map projections
Our old favourite... map projections.

## A quick refresher on projections
Location on Earth's surface is recorded in latitude and longitude.  Each is an angular measure relative to two imaginary references, the **Equator** (latitude = 0&deg;) and the **Prime Meridian** through Greenwich in London (longitude = 0&deg;). This system of geolocation makes sense because Earth is not flat (duh!) but a globe. The measurement system is shown in the figure below.

<img src='latlong.png' width="400">

Given a pair of coordinates we are accustomed to plotting them on $(x,y)$ axes, but when we do this with latitude and longitude coordinate pairs, we get ourselves in trouble, because although **parallels** (lines at equal latitude) are equally spaced from the equator to the poles, **meridians** converge toward the poles due to the (approximately) spherical shape of Earth.

The coordinate system we use to record location on Earth's three-dimensional surface can't be used directly to plot two-dimensional (flat) maps on paper or on screens. Or more correctly, it *can* be used but over any significant area of Earth surface substantial distortions will result.

To resolve this problem, we **project** latitude-longitude coordinates into other coordinate systems that have desired properties for particular mapping or analytical purposes.

This is a highly technical, yet fascinating area. We don't have time to get into it in any detail in this class, but but to be an effective geospatial professional, it is important to appreciate the implications. The most important of these is that we need to know projection is a thing!  The second is that **there is no perfect projection** just different projections that we might prefer for different purposes.  Of particular importance are **equal-area** projections, which preserve the relative area of regions on Earth surface and **conformal** projections which preserve shape.

Projected data are particularly useful because they can make calculation of distances and areas much simpler (we can apply simple geometry) than if we work with longitude-latitude coordinates.  For example, the equation for the distance between two locations with lon-lat coordinates $\left(\lambda_1,\phi_1\right)$ and $\left(\lambda_2,\phi_2\right)$ is given by (see [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula). 

$$ 
d=2 r \arcsin\left(\sqrt{\sin^2\left(\frac{\varphi_2 - \varphi_1}{2}\right) + \cos(\varphi_1) \cos(\varphi_2)\sin^2\left(\frac{\lambda_2 - \lambda_1}{2}\right)}\right)
$$

where $r$ is Earth's radius. On the other hand, if we work in projected $\left(x,y\right)$ coordinates in the plane, then over small areas of Earth's surface, particularly local maps we can use the much simpler

$$
d\approx\sqrt{(x_2-x_1)^2+(y_2-y_1)^2}
$$

This provides enormous computational benefits when we are calculating many distances and areas in carrying out spatial analysis.

## Projections in practice
OK. That's all very interesting, how do we handle projections in practice? 

Let's read two files of the same data, that are differently projected.

In [ ]:
counties1 = geopandas.read_file("ca-counties.shp")
counties2 = geopandas.read_file("ca-counties-LL.shp")

Now let's make two maps side by side for comparison. It's worth paying attention here to how we make the two panel figure. `plt.figure()` creates a `figure` object, which we add subplots to with the `add_subplot()` method. Each time we add a subplot, we get a reference to it by assigning the result to a variable (`ax1` and `ax2`). When we make the individual maps, we use `ax=ax1` or `ax=ax2` to determine which subplot we are drawing.

In [ ]:
fig = pyplot.figure()
fig.suptitle("California, two maps")

ax1 = fig.add_subplot(121)
counties1.plot(ax=ax1)

ax2 = fig.add_subplot(122)
counties2.plot(ax=ax2)

# There's a weird thing here where matplotlib produces
# text we are not interested in if we just use
#    ax1.set_title("counties1")
# We suppress the unwanted output by assigning the result to
# a throwaway variable
# To see how this changes things try removing "_ = "
_ = ax1.set_title("counties1")
_ = ax2.set_title("counties2")

Sure enough, they look different. We can find out what projections are in use in each case.

In [ ]:
counties1.crs, counties2.crs

To find out more about those, look them up at [epsg.io](https://epsg.io).

The important thing here is that these two datasets have projection information. That means it is possible to reproject them. For example, we can project `counties2` into the projection system of `counties1` like this.

In [ ]:
counties2_prj = counties2.to_crs(counties1.crs)
counties2_prj.plot()

Or, for that matter we can project the data to something else entirely. \[Also note here the use of `_ =` to suppress text output.\] 

In [ ]:
albers_proj4 = "+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
counties2_prj = counties2.to_crs(albers_proj4)
_ = counties2_prj.plot().set_title("Albers equal area, North America")

I found that **proj4** string at [epsg.io](https://epsg.io) and `geopandas` or more correctly the `pyproj` and `proj` libraries that are baked into it, can take it from there.

It's nice to know that once we have a dataset in a projection we like, it is easy to write it out to a file.

In [ ]:
counties2_prj.to_file("ca-counties-albers.shp")